In [4]:
import pandas as pd
import json

In [2]:
df = pd.read_csv(
    '/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/data/comparison_valid.csv'
)

In [3]:
df

,sample,annotation,ann
0,nasoic,"[(0, 6, 'O')]","[(0, 6, 'B-BRAND')]"
1,молочный коктейль банан,"[(0, 8, 'B-TYPE'), (9, 17, 'I-TYPE'), (18, 23,...","[(0, 8, 'B-TYPE'), (9, 17, 'I-TYPE'), (18, 23,..."
2,царские,"[(0, 7, 'B-TYPE')]","[(0, 7, 'B-BRAND')]"
3,зерница,"[(0, 7, 'B-TYPE')]","[(0, 7, 'B-BRAND')]"
4,cerkizov,"[(0, 8, 'O')]","[(0, 8, 'B-BRAND')]"
...,...,...,...
510,sarm,"[(0, 4, 'B-BRAND')]","[(0, 4, 'O')]"
511,каша жидкая безмолочная,"[(0, 4, 'B-TYPE'), (5, 11, 'O'), (12, 23, 'O')]","[(0, 4, 'B-TYPE'), (5, 11, 'I-TYPE'), (12, 23,..."
512,"кртки, вероки","[(0, 6, 'O'), (7, 13, 'O')]","[(0, 6, 'B-TYPE'), (7, 13, 'I-TYPE')]"
513,ipa,"[(0, 3, 'B-TYPE')]","[(0, 3, 'B-BRAND')]"


In [5]:
j = """{
        "category": {
            "id": 282,
            "title": "\u041e\u0442 \u041f\u0435\u0440\u0435\u043a\u0440\u0451\u0441\u0442\u043a\u0430",
            "slug": "ot-perekrestka",
            "parentId": "null"
        }
}"""

In [7]:
json.loads(j)["category"]

{'id': 282,
 'title': 'От Перекрёстка',
 'slug': 'ot-perekrestka',
 'parentId': 'null'}

In [9]:
with open("/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/parser/parse_perek_categories.json", "r") as f:
    cats = json.load(f)

In [10]:
len(cats)

30

In [21]:
for i in range(len(cats)):
    if cats[i]["category"]["title"] == "Молоко, сыр, яйца":
        print(cats[i]["category"]["id"])
        break


113


In [14]:
with open("/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/parser/parse_perek_Готовая еда_items.json", "r") as f:
    items = json.load(f)

In [15]:
len(items)

100

In [ ]:
from perekrestok_api import PerekrestokAPI
from perekrestok_api import abstraction

In [18]:
def main():
    with PerekrestokAPI() as Api:
        geopos_handler = Api.Geolocation.current()
        geopos = geopos_handler.json()
        print(f'Текущий город сессии {geopos["content"]["city"]["name"]} ({geopos["content"]["city"]["id"]})')

        # Получаем список категорий
        categories = Api.Catalog.tree()
        cat = categories.json()
        print(f'Список категорий: {len(cat["content"]["items"])}')
        with open('./parser/parse_perek_categories.json', 'w+', encoding="utf-8") as f:
            json.dump(cat["content"]["items"], f, indent=4, ensure_ascii=False) 

        # Выводим первую категорию
        cat_name = cat["content"]["items"][1]["category"]["title"]
        cat_id = cat["content"]["items"][1]["category"]["id"]
        print(f'Категория: {cat_name}, id: ({cat_id})')

        filter = abstraction.CatalogFeedFilter()
        filter.CATEGORY_ID = cat_id
        products = Api.Catalog.feed(filter=filter, page=1)
        response_items = products.json()

        with open(f'./parser/parse_perek_test_full_response.json', 'w+', encoding="utf-8") as f:
            json.dump(response_items, f, indent=4, ensure_ascii=False) 

In [25]:
df = pd.read_csv('../data/train.csv', sep=";")

In [28]:
df[['sample']].to_csv('../data/train_only_sample.csv', index=False)

## изучаем пятерочку

In [5]:
with open("/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/parser/5ka_catalog.json") as f:
    catalog = json.load(f)

In [6]:
len(catalog)

18

In [13]:
[cat['name'] for cat in catalog[2]['categories']]

['Овощи, зелень, грибы', 'Фрукты, ягоды', 'Орехи, сухофрукты']

In [9]:
len(catalog[11]['categories'])

6

In [11]:
catalog[11]['categories'][1]

{'id': '251C13043',
 'name': 'Газировка',
 'advert': None,
 'image_link': 'https://catalog-images.x5static.net/category/251C13043-icon-fivedelivery_a64e4d5a2818cc5f973fc74ea9c67bc0/345x345.png',
 'gradient_start': '#C9EEFE',
 'gradient_end': '#C9EEFE',
 'title_color': '#302E33',
 'additional_icons': [{'type': 'extended',
   'url': 'https://catalog-images.x5static.net/category/251C13043-additional-extended-fivedelivery_5a883d9d57bb156ff8fd94ea6bb28c98/312x348.png'},
  {'type': 'search_tree',
   'url': 'https://catalog-images.x5static.net/category/251C13043-additional-search-tree-fivedelivery_05359c735bc1bf7afb8b55c5706b89ab/144x144.png'}]}

In [12]:
with open("/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/parser/5ka_items_parsed.json") as f:
    items = json.load(f)

In [31]:
items['filters'][1]['list_values']['all'][:2]

['Черноголовка', 'Добрый']

In [33]:
items['products'][0]

{'plu': 1863,
 'name': 'Напиток Coca-Cola Classic 0.33л',
 'image_links': {'small': ['https://catalog-images.x5static.net/product/1157129-main/320x320.jpeg?updated_at=2025-02-14T15:15:56.083Z'],
  'normal': ['https://catalog-images.x5static.net/product/1157129-main/800x800.jpeg?updated_at=2025-02-14T15:15:56.083Z']},
 'uom': 'шт',
 'step': '1.0',
 'rating': {'rating_average': 4.84, 'rates_count': 5698},
 'promo': None,
 'prices': {'regular': '129.99', 'discount': None, 'cpd_promo_price': None},
 'labels': None,
 'property_clarification': '330 мл',
 'has_age_restriction': False,
 'stock_limit': None,
 'badges': [],
 'initial_weight_step': '1.0',
 'min_weight': '1.0',
 'orange_loyalty_points': 70,
 'is_available': True,
 'price_piece_unit': None}